## Get feature values
This notebook shows how feature values can be fetched from the onlione store with the python SDK.  
Make sure that the first data sync for each featureview has finished before requesting feature values.

In [ ]:
from google.api_core.exceptions import NotFound
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient,
    FeatureOnlineStoreServiceClient,
    NearestNeighborQuery,
    SearchNearestEntitiesRequest,
)
from google.cloud.aiplatform_v1beta1.types import (
    feature_online_store_service as feature_online_store_service_pb2,
)


def read_tfvars(file_path: str) -> dict[str, str]:
    """Reads terraform .tfvars file and returns a dictionary of key-value pairs."""
    tfvars = {}
    with open(file_path, "r") as f:
        for line in f:
            if "=" in line:
                key, value = line.strip().split("=")
                tfvars[key] = value.strip('"')
    return tfvars


vars = read_tfvars("infra/terraform.tfvars")


# Set variables
PROJECT = vars["gcp_project"]
REGION = vars["gcp_region"]
FEATUREONLINESTORE = (
    f"projects/{PROJECT}/locations/{REGION}/featureOnlineStores/example_online_store"
)
FEATUREVIEW_BASE = f"{FEATUREONLINESTORE}/featureViews/"
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

print(f"PROJECT: {PROJECT}")
print(f"REGION: {REGION}")

In [ ]:
admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)
try:
    PUBLIC_ENDPOINT = admin_client.get_feature_online_store(
        name=FEATUREONLINESTORE
    ).dedicated_serving_endpoint.public_endpoint_domain_name
    print(f"PUBLIC_ENDPOINT: {PUBLIC_ENDPOINT}")
except NotFound:
    print("Feature Online Store not found. Make sure to create it first.")
    exit()

In [ ]:
def sync_is_done(feature_view: str):
    """Check if at least one sync for the feature view has finished."""
    finished_syncs = [
        s
        for s in admin_client.list_feature_view_syncs(parent=feature_view)
        if s.run_time.HasField("end_time")
    ]
    if not finished_syncs:
        print("No finished syncs found.")
        return False
    return True

### Retrieve feature values by entity ID

In [ ]:
def get_feature_values_by_id(feature_view: str, entitiy_id: str):
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": API_ENDPOINT}
    )
    if not sync_is_done(feature_view):
        raise ValueError(
            "No data available for the feature view yet. Please wait for the first sync to finish."
        )

    return data_client.fetch_feature_values(
        feature_online_store_service_pb2.FetchFeatureValuesRequest(
            id=entitiy_id,
            feature_view=feature_view,
        )
    )

In [ ]:
feature_view_name = "user_featureview"
entity_id = "1"

response = get_feature_values_by_id(f"{FEATUREVIEW_BASE}{feature_view_name}", entity_id)
print(response)

In [ ]:
feature_view_name = "user_rating_featureview"
entity_id = "5"

response = get_feature_values_by_id(f"{FEATUREVIEW_BASE}{feature_view_name}", entity_id)
print(response)

### Search with embedding nearest neighbor

By entity ID


In [ ]:
def get_neighbors_by_id(feature_view: str, entitiy_id: str, k: int):
    # NOTE: the client uses the public endpoint here
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": PUBLIC_ENDPOINT}
    )
    if not sync_is_done(feature_view):
        raise ValueError(
            "No data available for the feature view yet. Please wait for the first sync to finish."
        )

    query = NearestNeighborQuery(entity_id=entitiy_id, neighbor_count=k)

    request = SearchNearestEntitiesRequest(feature_view=feature_view, query=query)
    return data_client.search_nearest_entities(request=request)

In [ ]:
feature_view_name = "movie_embedding_featureview"
feature_view = f"{FEATUREVIEW_BASE}{feature_view_name}"
embedding_dim = 1536

neighbors = get_neighbors_by_id(feature_view, "1", k=3)
print(neighbors)

By embedding

In [ ]:
def get_neighbors_by_embedding(feature_view: str, embedding: list[float], k: int):
    data_client = FeatureOnlineStoreServiceClient(
        client_options={"api_endpoint": PUBLIC_ENDPOINT}
    )
    if not sync_is_done(feature_view):
        raise ValueError(
            "No data available for the feature view yet. Please wait for the first sync to finish."
        )
    
    query = NearestNeighborQuery(
        embedding=NearestNeighborQuery.Embedding(value=embedding), neighbor_count=k
    )
    request = SearchNearestEntitiesRequest(feature_view=feature_view, query=query)
    return data_client.search_nearest_entities(request=request)

In [ ]:
neighbors = get_neighbors_by_embedding(feature_view, [0.1] * embedding_dim, k=3)
print(neighbors)